<a href="https://colab.research.google.com/github/LeoProvorov/Data-science/blob/master/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%D1%82%D0%B5%D0%BD%D0%B4%D0%B5%D0%BD%D1%86%D0%B8%D0%B9%20%D0%BD%D0%B0%20%D1%80%D1%8B%D0%BD%D0%BA%D0%B5%20%D1%82%D1%80%D1%83%D0%B4%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Трек #1 Анализ тенденций на рынке труда в условиях пандемии

**Цель исследования:** определить портреты наиболее уязвимых групп граждан и проследить динамику безработицы среди этих групп, чтобы предложить рекомендации по поддержке этих категорий граждан


**Группы:** 
1.   многодетные и неполные семьи
2.   люди предпенсионного возраста
3.   малоимущие граждане, заключившие социальный контракт


**Фокус исследования**: выделены 3 категории лиц, испытывающих трудности

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
import math as mth
import matplotlib.pyplot as plt
import requests
from io import StringIO
pd.set_option('display.max_columns', None)

In [ ]:
# настройка вывода данных на экран
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 800)
pd.set_option('display.expand_frame_repr', False)
pd.options.display.float_format = '{:,.2f}'.format

# Выводить по умолчанию график нужного размера
plt.rcParams['figure.figsize'] = (16,8)

In [ ]:
# импорт данных

url = 'https://drive.google.com/file/d/1LdUOUmHFqAULtwujV4FOHKq-IKlSN2JH/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path, sep=";", encoding = "utf8")


In [ ]:
df.head(2)

,id,month_unemployed,week_unemployed,gender,age,hc_any,hc_repeat,hc_didnotwork,hc_predpens,hc_largefam,hc_finisheducczn,hc_divorced,hc_singleparent,hc_redundantworkers,hc_socialcontr,jobless_category,month_jobless,month_end_busyness,education,year_graduation,profession_last_educ,region,district,experience,salary_average,profession_last_work,dismissal_reason,month_dismissal,educ_napravl,education_speciality,prof_education_dur,work_napravl_number,work_napravl_refusal_number,work_refusal_number,employer_refusal_number,expected_salary,employment_type,employment_mode,month_employment,week_employment,profession_employment,month_close,reason_close,month_end,reason_end
0,3d7cb8ae-a038-4a8d-af40-4529cf532426,2020-03,11,Женский,4.00,False,False,False,False,False,False,False,False,False,False,"Уволенный из организации, ИП, с муниципальной или государственной службы",-1,2020-03,Высшее образование - специалитет,2011,Экономист,Республика Татарстан (Татарстан),Приволжский федеральный округ,9.00,24000.0,Специалист,"Расторжение трудового договора по инициативе работника, служащего",2020-01,False,-1,-1,1.0,0.0,1.0,1.0,25000.0,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,a8bd7bf2-c47d-4391-bffc-7e8275b22781,2020-07,27,Мужской,4.00,True,False,True,False,False,False,False,False,False,False,-1,-1,2020-07,Среднее общее образование,-1,-1,Республика Татарстан (Татарстан),Приволжский федеральный округ,0.00,-1,-1,-1,-1,False,-1,-1,2.0,0.0,2.0,2.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
# заменяем все пропуски (None) на -1 для удобства обработки
df = df.replace("None", "-1")

In [ ]:
#преобразуем возраст в понятные величины
meaning = {1:'16-17',
          2:'18-22',
          3:'23-27',
          4:'28-32',
          5:'33-37',
          6:'38-42',
          7:'43-47',
          8:'48-52',
          9:'53-54',
          10:'55',
          11:'56',
          12:'57',
          13:'58',
          14:'59',
          15:'60',
          16:'старше 60'}

In [ ]:
df['age'] = df['age'].replace(meaning)

In [ ]:
# в данных с заработными платами удаляем пробелы и преобразуем текстовые данные в числовые
salary = ['expected_salary','salary_average']
df[salary] = df[salary].replace(r'\s+','',regex=True)

df['salary_average'] = pd.to_numeric(df['salary_average']).astype('int')
df['expected_salary'] = pd.to_numeric(df['expected_salary']).astype('int')   

In [ ]:
# выделяем отдельные датасеты по анализируемым группам и добавляем колонку с этим признаком: 
# лица предпенсионного возраста 
gr_predpens = df.query('hc_predpens == True')
gr_predpens['group'] = "predpens"
# многодетные и неполные семьи
gr_kids = df.query('hc_largefam	 == True or hc_singleparent == True')
gr_kids['group'] = "kids"
# малоимущие граждане, заключивщие социальный контракт
gr_social = df.query('hc_socialcontr == True')
gr_social['group'] = "social"

C:\Anaconda_3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda_3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Anaconda_3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [ ]:
# объединяем вырезанные датасеты в один
df2 = pd.concat([gr_predpens, gr_kids, gr_social])

In [ ]:
# готовим сводные данные по всем безработным 
df['group'] = "total"
# группируем количество безработных по региону, полу, возрасту и т.д.
svod_total = df.pivot_table(index=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],
                values='id',aggfunc='count').reset_index()
# так же группируем выделенные по группам лиц данные
svod = df2.pivot_table(index=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],
                values='id',aggfunc='count').reset_index()
# объединяем сгруппиованные данные
svod = pd.concat([svod, svod_total])

In [ ]:
# выделяем данные по зарплатам для расчета средних показателей
salary_1 = df2.query('expected_salary > 0')
salary_2 = df2.query('salary_average > 0')
salary_1_total = df.query('expected_salary > 0')
salary_2_total = df.query('salary_average > 0')

In [ ]:
# группируем данные по средним зарплатам в разрезе региона, пола, возраста и т.д.
svod_1 = salary_1.pivot_table(index=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],
                values='expected_salary',aggfunc='median').reset_index()

svod_2 = salary_2.pivot_table(index=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],
                values='salary_average',aggfunc='median').reset_index()
svod_1_total = salary_1_total.pivot_table(index=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],
                values='expected_salary',aggfunc='median').reset_index()

svod_2_total = salary_2_total.pivot_table(index=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],
                values='salary_average',aggfunc='median').reset_index()

In [ ]:
# выделяем данные с гражданами, нашедшими работу
employ = df2.query('month_employment != "-1"')
employ_total = df.query('month_employment != "-1"')

In [ ]:
# Группируем данные с трудоустроенными гражданами в разрезе региона, пола, возраста и др.
svod_3 = employ.pivot_table(index=['group', 'district', 'region', 'month_employment', 'gender', 'education', 'age', 'hc_didnotwork'],
                values='id',aggfunc='count').reset_index()
svod_3_total = employ_total.pivot_table(index=['group', 'district', 'region', 'month_employment', 'gender', 'education', 'age', 'hc_didnotwork'],
                values='id',aggfunc='count').reset_index()

In [ ]:
# объединяем сгруппированные данные
svod_1 = pd.concat([svod_1, svod_1_total])
svod_2 = pd.concat([svod_2, svod_2_total])

In [ ]:
# объединяем сгруппированные данные
svod_3 = pd.concat([svod_3, svod_3_total])
svod_3.columns = ['group', 'district', 'region', 'month', 'gender', 'education', 'age', 'hc_didnotwork', 'employment amount']

In [ ]:
# объединяем сгруппированные данные в общий датасет
svod = svod.merge(svod_1, left_on=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],
          right_on=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],how='left')

In [ ]:
# объединяем сгруппированные данные в общий датасет
svod = svod.merge(svod_2, left_on=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],
          right_on=['group', 'district', 'region', 'month_unemployed', 'gender', 'education', 'age', 'hc_didnotwork'],how='left')

In [ ]:
# переименовываем заголовки столбцов
svod.columns = ['group', 'district', 'region', 'month', 'gender', 'education', 'age', 'hc_didnotwork', 'amount', 'expected_salary', 'salary_average']

In [ ]:
# объединяем сгруппированные данные в общий датасет
svod = svod.merge(svod_3, left_on=['group', 'district', 'region', 'month', 'gender', 'education', 'age', 'hc_didnotwork'],
          right_on=['group', 'district', 'region', 'month', 'gender', 'education', 'age', 'hc_didnotwork'],how='left')

In [ ]:
# приводим количество граждан в числовой формат
svod['employment amount']  = svod['employment amount'] * -1
svod['employment amount'] = svod['employment amount'].fillna(0)
svod['employment amount'] = svod['employment amount'].astype('int64')

In [ ]:
svod.head()

,group,district,region,month_unemployed,gender,education,age,hc_didnotwork,amount,expected_salary,salary_average
0,kids,Приволжский федеральный округ,Кировская область,2020-03,Женский,Высшее образование - бакалавриат,23-27,False,1,15000.0,19000.0
1,kids,Приволжский федеральный округ,Кировская область,2020-03,Женский,Высшее образование - специалитет,28-32,False,1,15000.0,19000.0
2,kids,Приволжский федеральный округ,Кировская область,2020-03,Женский,Высшее образование - специалитет,33-37,False,4,27500.0,11000.0
3,kids,Приволжский федеральный округ,Кировская область,2020-03,Женский,Высшее образование - специалитет,38-42,False,2,13500.0,14500.0
4,kids,Приволжский федеральный округ,Кировская область,2020-03,Женский,Высшее образование - специалитет,43-47,False,2,14000.0,24000.0


In [ ]:
svod.isna().mean()

In [ ]:
svod['education'] = svod['education'].replace('Не указано', '-1')

In [ ]:
# переведём ожидаемую зп в int (пропуски заменим на 0)
svod['expected_salary'] = svod['expected_salary'].fillna(0)
svod['expected_salary'] = svod['expected_salary'].replace('не указано', 0)

In [ ]:
svod['expected_salary'] = pd.to_numeric(svod['expected_salary']).astype('int')
svod.head(2)

In [ ]:
# доля пропусков в образовании составляет около 1.2%
svod[svod['education'] == '-1'].shape[0] / svod.shape[0]

In [ ]:
# добавим группы образования

def edu_group(row):
    if row['education'] in ['Высшее образование - бакалавриат',
                            'Высшее образование - специалитет',
                            'Высшее образование - магистратура',
                            'Высшее образование - подготовка кадров высшей квалификации']:
        return 'Высшее'
    elif row['education'] in ['Основное общее образование',
                              'Среднее общее образование',
                              'Начальное общее образование']:
        return 'Среднее'
    elif row['education'] in ['Среднее профессиональное образование']:
        return 'Среднее профессиональное'
    elif row['education'] in ['Другое', '-1']: # незаполненное относим в группе ДРУГОЕ !!!!!!!
        return 'Другое'
    else:
        return '!!! ОШИБКА !!!'


In [ ]:
svod['edu_group'] = svod.apply(edu_group, axis=1)

svod.head(3)

In [ ]:
# переименуем "-1" в образовании на "не указано"
svod['education'] = svod['education'].replace('-1', 'Не указано')

In [ ]:
svod[svod['edu_group'] == '!!! ОШИБКА !!!'].head(3)
# ошибок нет

In [ ]:
# переведём среднюю зп в int (пропуски заменим на median по региону)
svod['salary_average'] = svod['salary_average'].replace('не указано', np.nan)

salary_avg_nonan = svod[svod['salary_average'] != np.nan]
salary_avg_nonan['salary_average'] = pd.to_numeric(salary_avg_nonan['salary_average']).astype('float')
salary_avg_nonan['salary_average'] = salary_avg_nonan['salary_average'].round()
salary_avg_nonan.head(2)


In [ ]:
salary_reg_median = salary_avg_nonan.groupby(['region', 'gender', 'edu_group']) \
    ['salary_average'].median().reset_index()

salary_reg_median.head()

In [ ]:
if 'salary_average_x' in svod.columns:
    pass
else:
    svod = pd.merge(svod, 
                  salary_reg_median, 
                  on=['region', 'gender', 'edu_group'], 
                  how='left')
svod.head(3)

In [ ]:
svod.shape[0]

In [ ]:
svod[svod['salary_average_x'].isna()].shape[0]

In [ ]:
svod.loc[svod['salary_average_x'].isna(), 'salary_average_x'] \
    = svod.loc[svod['salary_average_x'].isna(), 'salary_average_y']

In [ ]:
svod[svod['salary_average_x'].isna()]

In [ ]:
if svod[svod['salary_average_x'].isna()].shape[0] > 0:
    svod = svod.drop(index=svod[svod['salary_average_x'].isna()].index)
else:
    pass

In [ ]:
if 'salary_average_x' in svod.columns:
    svod['salary_average_x'].unique()
    svod['salary_average_x'].isna().mean()
    svod = svod.drop('salary_average_y', axis=1).rename(columns={'salary_average_x':'salary_average'})
else:
    pass
svod['salary_average'] = pd.to_numeric(svod['salary_average']).astype('int')

In [ ]:
# Разберёмся с датами
svod['month'] = pd.to_datetime(svod['month'], format='%Y-%m-%d')
svod['month'].dtype

In [ ]:
svod.to_csv(r'C:\Users\Пользователь\Documents\svod_volga.csv', index = False)

#Дополнительная информация по пандемии

Ввод ограничений в стране по датам выступления президента:  
25.03 – объявлены выходные дни с субботы 28 марта по воскресенье 5 апреля  
02.04 - нерабочие дни продлены до конца месяца, то есть по 30 апреля включительно  
28.04 -  период нерабочих дней продлен до 11 мая включительно  
11.05 – с 12.05 единый период нерабочих дней для всей страны завершился. Далее поэтапный выход из режима ограничений в каждом регионе по-своему.

## Пособия
### Апрель  

1. До конца 2020 года максимальная сумма увеличена с 8000 до 12 130 рублей в месяц - **Постановление от 27.03**
2. Уволенным с 1 марта 2020 года пособие за апрель, май, июнь назначается в максимальном размере  - **12.04**
3. Одному из родителей, уволенному с 1 марта 2020 года, в апреле, мае и июне назначается доплата в размере 3000 рублей на каждого ребенка.
4. Встать на учет в качестве безработного можно дистанционно (упрощенный порядок) - **9.04**

### Июнь (постановление от 10.06)
1. Минимальный размер пособия в мае — июле 2020 года составляет 4500 рублей в месяц. В другие месяцы минимальный размер пособия составит 1500 рублей. 
2. В июне — августе 2020 года одному из родителей, который зарегистрирован в качестве безработного, дополнительно выплачиваются по 3000 рублей на каждого ребенка до 18 лет. 
3. Индивидуальные предприниматели, прекратившие свою деятельность после 1 марта и признанные безработными, смогут получать пособие в размере 12 130 рублей. Эта сумма назначается на срок не более трех месяцев, но не позднее 1 октября 2020 года.
4. Безработным гражданам, у которых истек срок выплаты пособия после 1 марта 2020 года, выплата автоматически продлевается на срок не более трех месяцев, но не позднее 1 октября 2020 года. Размер нового пособия будет равен той сумме, которая выплачивалась ранее, но в пределах максимального и минимального размера, установленного на день продления выплаты.

Июль (Постановление от 4.07)
Продлен срок выплат пособия по безработице в максимальном размере:
1. Уволенным с 1 марта 2020 года будет выплачиваться пособие в размере 12 130 рублей в месяц не только с апреля по июнь, но и в июле и августе.
2. Минимальный размер пособия повышен до 4500 рублей не только в период с мая по июль, но также и в августе 2020 года.
Сентябрь (Постановление от 9.09)
Срок доплаты безработным родителям продлен по сентябрь включительно. Один из родителей сможет получать по 3000 ₽ на каждого несовершеннолетнего ребенка.
